In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from itertools import combinations, chain
from collections import defaultdict

## 1. Datasets

### Utility Matrix

In [2]:
# load utility matrix
utility_matrix = pd.read_csv('./data/sparse_matrix.csv', index_col=0)
utility_matrix

,78_1,78_2,78_3,78_4,78_5,78_6,78_7,78_8,78_9,78_10,...,32_13,32_14,32_15,32_16,32_17,32_18,radiant_win,radiant_team_id,dire_team_id,team
7750912161,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,9425660.0,8629317.0,1.0
7750914469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,8629005.0,8629318.0,1.0
7750915644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,8961813.0,9330489.0,0.0
7750937564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,9344594.0,9395679.0,0.0
7750968496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,9330489.0,8961813.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7881636100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,8970060.0,8957156.0,1.0
7881664207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,True,8957156.0,8970060.0,1.0
7881677439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,9255039.0,2163.0,0.0
7881696382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,8849990.0,8936613.0,0.0


### Transaction db

In [3]:
# create transaction db matrix from picks_bans
db = pd.read_csv('./data/picks_bans.csv')
db

,hero_id,team,order,match_id
0,78.0,1.0,0.0,7750912161
1,95.0,0.0,1.0,7750912161
2,51.0,0.0,2.0,7750912161
3,9.0,1.0,3.0,7750912161
4,63.0,0.0,4.0,7750912161
...,...,...,...,...
146108,94.0,1.0,19.0,7881723710
146109,20.0,1.0,20.0,7881723710
146110,70.0,0.0,21.0,7881723710
146111,7.0,0.0,22.0,7881723710


In [4]:
# use order to filter picks per team
order_picks = [7, 8, 12, 13, 14, 15, 16, 17, 22, 23]
db = db[db['order'].isin(order_picks)].groupby(['match_id', 'team'])['hero_id'].apply(set).tolist()
db

[{33.0, 74.0, 98.0, 107.0, 136.0},
 {2.0, 21.0, 27.0, 31.0, 59.0},
 {17.0, 19.0, 48.0, 57.0, 129.0},
 {10.0, 11.0, 84.0, 86.0, 91.0},
 {29.0, 60.0, 90.0, 107.0, 123.0},
 {11.0, 28.0, 41.0, 53.0, 111.0},
 {22.0, 53.0, 94.0, 104.0, 112.0},
 {1.0, 3.0, 40.0, 81.0, 90.0},
 {39.0, 42.0, 51.0, 119.0, 135.0},
 {17.0, 31.0, 53.0, 91.0, 97.0},
 {8.0, 25.0, 60.0, 100.0, 138.0},
 {19.0, 46.0, 53.0, 54.0, 87.0},
 {50.0, 63.0, 86.0, 97.0, 114.0},
 {3.0, 14.0, 15.0, 29.0, 110.0},
 {6.0, 41.0, 46.0, 62.0, 102.0},
 {17.0, 40.0, 73.0, 75.0, 120.0},
 {10.0, 21.0, 46.0, 98.0, 135.0},
 {3.0, 14.0, 39.0, 49.0, 72.0},
 {7.0, 40.0, 49.0, 97.0, 98.0},
 {19.0, 27.0, 45.0, 93.0, 104.0},
 {11.0, 20.0, 49.0, 88.0, 97.0},
 {13.0, 27.0, 68.0, 78.0, 113.0},
 {14.0, 67.0, 86.0, 108.0, 120.0},
 {41.0, 73.0, 75.0, 129.0, 136.0},
 {12.0, 39.0, 60.0, 86.0, 119.0},
 {7.0, 34.0, 79.0, 114.0, 129.0},
 {9.0, 10.0, 34.0, 97.0, 107.0},
 {20.0, 22.0, 33.0, 44.0, 51.0},
 {7.0, 8.0, 62.0, 74.0, 120.0},
 {2.0, 14.0, 27.0, 114.0, 1

In [5]:
len(db)

12180

## 2. Helper Functions

In [6]:
def cosine_similarity(a, b):
    # YOUR CODE HERE
    common = a.notna() & b.notna()
    u_common = a[common]
    v_common = b[common]
    if len(u_common) > 0:
        return np.dot(u_common, v_common) / (np.linalg.norm(u_common) * np.linalg.norm(v_common))
    return 0


def fpgrowth(db, minsup):
    # YOUR CODE HERE
    def build_fptree(db):
        counts = {
            item: count
            for item, count in Counter(chain.from_iterable(db)).items()
            if count >= minsup
        }
        if not counts:
            return None, None
            
        header = defaultdict(list)
        root = {'count': 0, 'children': {}, 'parent': None}
        for transaction in db:
            sorted_items = sorted([item for item in transaction if item in counts], key=lambda item: (-counts[item], item))
            current_node = root
            for item in sorted_items:
                if item not in current_node['children']:
                    new_node = {'count': 0, 'children': {}, 'parent': current_node}
                    current_node['children'][item] = new_node
                    header[item].append(new_node)
                
                current_node = current_node['children'][item]
                current_node['count'] += 1

        return root, header
    
    def mine_fptree(header, prefix):
        result = []
        items = sorted(header.items(), key=lambda x: (sum(node['count'] for node in x[1]), x[0]))
        
        for item, nodes in items:
            new_freqset = prefix + [item]
            support = sum(node['count'] for node in nodes)
            result.append((tuple(sorted(new_freqset)), support))
            conditional_base = []
            for node in nodes:
                path = []
                current = node['parent']
                while current and current['parent'] is not None:
                    for parent_item, parent_node in current['parent']['children'].items():
                        if parent_node is current:
                            path.append(parent_item)
                            break
                    current = current['parent']
                
                if path:
                    path.reverse()
                    conditional_base.append((path, node['count']))
                    
            if conditional_base:
                subtree, subheader = build_fptree([path for path, count in conditional_base for _ in range(count)])
                if subheader:
                    result.extend(mine_fptree(subheader, new_freqset))

        return result
    
    root, header = build_fptree(db)
    if not header:
        return []
    
    freq_itemsets = mine_fptree(header, [])
    return sorted(freq_itemsets, key=lambda x: (-x[1], -len(x[0]), x[0]))
    # raise NotImplementedError()


def assoc(fi, minconf, db_size):
    # YOUR CODE HERE
    support_dict = dict(fi)
    rules = [
        {
            'antecedent': tuple(sorted(antecedent)),
            'consequent': consequent[0],
            'support': support,
            'confidence': conf,
            'lift': conf / (relSup / db_size)
        }
        for itemset, support in fi if len(itemset) > 1
        for antecedent in chain.from_iterable(combinations(itemset, r) for r in range(1, len(itemset)))
        if len((consequent := tuple(sorted(set(itemset) - set(antecedent))))) == 1
        if (antecedent_support := support_dict.get(tuple(sorted(antecedent)), 0)) > 0
        if (conf := support / antecedent_support) >= minconf
        if (relSup := support_dict.get(consequent, 0)) > 0
    ]

    return sorted(rules, key=lambda x: (-x['lift'], -x['confidence'], -x['support'], x['consequent'], x['antecedent']))
    # raise NotImplementedError()

## 3. FIM

In [10]:
fi_fpgrowth = fpgrowth(db, 2)
rules = assoc(fi_fpgrowth,
    0.2,
    2,
)

In [11]:
rules

[{'antecedent': (5.0, 13.0, 46.0, 86.0),
  'consequent': 80.0,
  'support': 2,
  'confidence': 1.0,
  'lift': 0.025},
 {'antecedent': (45.0, 100.0, 110.0),
  'consequent': 113.0,
  'support': 2,
  'confidence': 0.6666666666666666,
  'lift': 0.024691358024691357},
 {'antecedent': (9.0, 55.0, 86.0),
  'consequent': 82.0,
  'support': 2,
  'confidence': 1.0,
  'lift': 0.024390243902439025},
 {'antecedent': (18.0, 22.0, 49.0),
  'consequent': 57.0,
  'support': 2,
  'confidence': 0.4,
  'lift': 0.022857142857142857},
 {'antecedent': (14.0, 22.0, 27.0, 46.0),
  'consequent': 12.0,
  'support': 2,
  'confidence': 1.0,
  'lift': 0.02197802197802198},
 {'antecedent': (19.0, 27.0, 74.0),
  'consequent': 32.0,
  'support': 2,
  'confidence': 0.6666666666666666,
  'lift': 0.02185792349726776},
 {'antecedent': (60.0, 64.0, 107.0),
  'consequent': 67.0,
  'support': 2,
  'confidence': 1.0,
  'lift': 0.02127659574468085},
 {'antecedent': (17.0, 20.0, 46.0, 114.0),
  'consequent': 23.0,
  'support': 

In [8]:
fim.apriori(db)

ValueError: invalid evaluation measure

## 4. Run Simulations